In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F 
# udf, col, year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1644538333842_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read CSV into Spark Dataframe

In [28]:
df_btc = spark.read.csv('s3://patrickudacityde/Capstone_project/e-5Z34PVB9W57QX1CTO1QTRKD0V/Data/h5/spot/BTC-USDT_15m.csv',
                   header = True)
df_eth = spark.read.csv('s3://patrickudacityde/Capstone_project/e-5Z34PVB9W57QX1CTO1QTRKD0V/Data/h5/spot/ETH-USDT_15m.csv',
                   header = True)
df_eos = spark.read.csv('s3://patrickudacityde/Capstone_project/e-5Z34PVB9W57QX1CTO1QTRKD0V/Data/h5/spot/EOS-USDT_15m.csv',
                   header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_btc.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------+--------+--------+--------+----------+
|_c0|   candle_begin_time|    open|    high|     low|   close|    volume|
+---+--------------------+--------+--------+--------+--------+----------+
|  0|2018-01-01 00:00:...|13715.65|13715.65|13400.01|13556.15|123.616013|
|  1|2018-01-01 00:15:...|13533.75|13550.87| 13402.0|13521.12|  98.13643|
|  2|2018-01-01 00:30:...| 13500.0|13545.37| 13450.0|13470.41| 79.904037|
|  3|2018-01-01 00:45:...|13494.65|13690.87| 13450.0|13529.01|141.699719|
|  4|2018-01-01 01:00:...|13528.99|13571.74|13402.28|13445.63| 72.537533|
|  5|2018-01-01 01:15:...|13469.99|13595.89|13445.63| 13560.0| 87.861758|
|  6|2018-01-01 01:30:...|13559.99|13559.99| 13430.0|13440.01| 74.368405|
|  7|2018-01-01 01:45:...|13440.01|13440.01|13155.38|13203.06| 148.92931|
|  8|2018-01-01 02:00:...| 13203.0|13411.76|13202.03| 13387.0| 108.54247|
|  9|2018-01-01 02:15:...| 13387.0| 13388.0| 13225.0| 13225.0| 95.186039|
+---+--------------------+--------+---

# Data Count/size

In [35]:
for parquet in [df_btc,df_eth,df_eos]:
    print(parquet.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

143091
143091
129101

# Data Quality check 

In [33]:
# check missing value and drop the rows with missing value
empty_data_info = []
for parquet in [df_btc,df_eth,df_eos]:
    for col in parquet.columns:
        if parquet.filter(F.col(col).isNull()).count() > 0:
            empty_data_info.append([col,'has null value'])
            parquet = parquet.filter(F.col(col).isNotNull())
            

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
empty_data_info
# NO empty entry

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [43]:
# check duplicate entry

for parquet in [df_btc,df_eth,df_eos]:
    if parquet.groupBy('candle_begin_time').count().filter("count >1").count() > 0:
        print ("{} has duplicate records".format(parquet))
    else: print ("no duplicate entry")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

no duplicate entry
no duplicate entry
no duplicate entry

# Save Parquet

In [ ]:

for t in ['BTC-USDT_15m.csv','ETH-USDT_15m.csv','EOS-USDT_15m.csv']:
    df = spark.read.csv('s3://patrickudacityde/Capstone_project/e-5Z34PVB9W57QX1CTO1QTRKD0V/Data/h5/spot/{}'\
                        .format(t),header = True)
    empty_data_info = []
    for parquet in [df_btc,df_eth,df_eos]:
        for col in parquet.columns:
            if parquet.filter(F.col(col).isNull()).count() > 0:
                empty_data_info.append([col,'has null value'])
                parquet = parquet.filter(F.col(col).isNotNull())
                logger.write(empty_data_info)
    for parquet in [df_btc,df_eth,df_eos]:
        if parquet.groupBy('candle_begin_time').count().filter("count >1").count() > 0:
            print ("{} has duplicate records".format(parquet))
        else: print ("no duplicate entry")
    spark.write.parquet("s3://patrickudacityde/Capstone_project/e-5Z34PVB9W57QX1CTO1QTRKD0V/Data/h5/spot/{}"\
                       .format(t.replace(".csv","")))